# Creating the initial position file
The following code read the pour points extracted from Arc gis from the intersection of watersheds with lake huron boundary.
 We are using PyLag Create initial position function to Create a release Zone for each of the Pour points. Before creating the initial positions, We want to add a condition to make a the relase zone that are in the boundary of Lake Huron.
 
 Then using Pylag create initial position file it will create a release zone for each pour point/groupid, in the end each group_id will contain sets of release zone 
 6/3/2024

#### Part 1-1: Import required libraries

In [1]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, box
import matplotlib.pyplot as plt
from pylag.processing.coordinate import get_epsg_code, utm_from_lonlat, lonlat_from_utm
from pylag.processing.coordinate import utm_from_lonlat, lonlat_from_utm
from pylag.processing.release_zone import create_release_zones_along_cord
from pylag.processing.plot import create_figure, colourmap
from pylag.processing.plot import FVCOMPlotter
from pylag.processing.release_zone import create_release_zone
from pylag.processing.input import create_initial_positions_file_multi_group
from pylag.processing.input import create_initial_positions_file_single_group

#### Part1-2: Read the initial position file(from watreshed and lake intersections) and rename the columns

In [2]:
# define the path to the input file
data_dir ='/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/input/initial_position'
init_file = os.path.join(data_dir, 'WatershedPourPoints_LH_copy_multigroup.dat')

cwd = os.getcwd()
# Create run directory
simulation_dir = '/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron'.format(cwd)
try:
    os.makedirs(simulation_dir)
except FileExistsError:
    pass

df = pd.read_csv(init_file, skiprows=0,sep=' ',header=None)
# rename the columns
df.rename(columns={0:"group_id", 1:"lon",2:"lat",3:"depth"}, inplace=True)

# Part2: Create a release zone and save it

In [3]:
# Release zone  
radius = 0.01
# target number of particles
n_particles_target = 100

# Release depths
depth_below_surface = 0.0
# list to accumulate release zones
release_zones = []

# Loop through the rows of the DataFrame
for index, row in df.iterrows():
    group_id = row['group_id']
    lat = row['lat']
    lon = row['lon']
    
    # Create the release zone
    surface_release_zone = create_release_zone(group_id = group_id,
                                           radius = radius,
                                           centre = [lon, lat],
                                           n_particles = n_particles_target,
                                           depth = depth_below_surface,
                                           random = False)
    
    # accumulate the release zones
    release_zones.append(surface_release_zone)
    
    
 # Create input sub-directory
input_dir = os.path.join(simulation_dir, 'input/initial_position')
os.makedirs(input_dir, exist_ok=True)

    # Output filename
file_name = os.path.join(input_dir, 'WatershedPourPoints_LH_multigroup_3.dat')
# Write data to file
create_initial_positions_file_multi_group(file_name, release_zones)

# Insure the points are placed inside Lake

In this part I save the initial position file as shape file and I used arcpy to remove the points that are not inside Lake Huron. 

Code for removing the points is in DirectNutrientLoad directory and it's called Initial_position_removeoutsidelakepoints

In [4]:
# read file name
df = pd.read_csv(file_name, skiprows=1,sep=' ',header=None)
# rename the columns
df.rename(columns={0:"group_id", 1:"lon",2:"lat",3:"depth"}, inplace=True)
df.head()

,group_id,lon,lat,depth
0,0.0,275.334159,45.735061,0.0
1,0.0,275.328159,45.737061,0.0
2,0.0,275.330159,45.737061,0.0
3,0.0,275.332159,45.737061,0.0
4,0.0,275.334159,45.737061,0.0


In [5]:
df['lon'] = df['lon'] -360
df

,group_id,lon,lat,depth
0,0.0,-84.665841,45.735061,0.0
1,0.0,-84.671841,45.737061,0.0
2,0.0,-84.669841,45.737061,0.0
3,0.0,-84.667841,45.737061,0.0
4,0.0,-84.665841,45.737061,0.0
...,...,...,...,...
10687,147.0,-83.465788,43.746634,0.0
10688,147.0,-83.463788,43.746634,0.0
10689,147.0,-83.461788,43.746634,0.0
10690,147.0,-83.459788,43.746634,0.0


In [6]:
# save the dataframe to shape_file called WatershedPourPoints_LH_multigroup_rounded.shp
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
gdf.crs = 'EPSG:4326'
gdf.to_file('/mnt/d/Users/abolmaal/Arcgis/NASAOceanProject/GIS_layer/particle_tracking_output/WatershedPourPoints_LH_multigroup_rounded_initialposition2.shp')

# 3-Solve an error related to PyLag

My Lat and lon number has more than 10 digits and when I run it through Pylag it gives me this error:ValueError: invalid literal for int() with base 10: '0.0' so to avoid this I want to read Lat and Lon and round them by 10


Once I removed the points outside Lake Boundary I read the cleaned point here again 
- make sure Longitudes are in 360 degree 
- Lat and Lon are rounded to 10 
- Save the file as dat file



In [ ]:
# read the following shapefile output_path = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\particle_tracking_output\WatershedPourPoints_LH_multigroup_rounded_initialposition_intersection.shp"
output_path = r"/mnt/d/Users/abolmaal/Arcgis/NASAOceanProject/GIS_layer/particle_tracking_output/WatershedPourPoints_LH_multigroup_rounded_initialposition_intersection2.shp"
updated_gdf = gpd.read_file(output_path)
# updated_gdf[lon] = updated_gdf[lon] + 360
updated_gdf['lon'] = updated_gdf['lon'] + 360

# save the updated_gdf as a dat file and keep only the group_id, lon, lat, and depth columns
updated_gdf = updated_gdf[['group_id', 'lon', 'lat', 'depth']]
# sort the data from group_id
updated_gdf = updated_gdf.sort_values(by=['group_id'])
# if df[lon] and df[lat] are greater than 10 digits round them to 10 digits
updated_gdf['lon'] = updated_gdf['lon'].round(10)
updated_gdf['lat'] = updated_gdf['lat'].round(10)
updated_gdf

,group_id,lon,lat,depth
4911,0.0,275.334159,45.755061,0.0
4887,0.0,275.328159,45.749061,0.0
4886,0.0,275.326159,45.749061,0.0
4885,0.0,275.342159,45.747061,0.0
4884,0.0,275.340159,45.747061,0.0
...,...,...,...,...
2351,147.0,276.528212,43.742634,0.0
2350,147.0,276.526212,43.742634,0.0
2338,147.0,276.536212,43.740634,0.0
2336,147.0,276.532212,43.740634,0.0


In [7]:
# save the updated_gdf as a dat file and skip the header
updated_gdf.to_csv('/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/input/initial_position/WatershedPourPoints_LH_multigroup_rounded_initialposition_intersection2.dat', sep=' ', index=False, header=False)